In [ ]:
import os
from typing import List, Literal, Union

import cv2
import numpy as np
import pandas as pd
from edesdetectrl import dataloaders
from edesdetectrl.config import config
import edesdetectrl.dataloaders.echonet.label_frames as lf
from edesdetectrl.dataloaders.echonet import _get_filenames

split = "TRAIN"
filelist_csv_file = config["data"]["filelist_path"]
volumetracings_df = pd.read_csv(
    config["data"]["volumetracings_path"], index_col="FileName"
)
videos_dir = config["data"]["videos_path"]

filenames = _get_filenames(filelist_csv_file, split)

In [ ]:
filenames[0]

In [ ]:
def loadvideo(filename: str) -> np.ndarray:
    # Copied and modified from https://github.com/echonet/dynamic/blob/f6d9b342ffd6e5129bcf15a994e1734fc31003f6/echonet/utils/__init__.py#L16
    """Loads a video from a file.
    Args:
        filename (str): filename of video
    Returns:
        A np.ndarray with dimensions (frames, height, width). The values
        will be uint8's ranging from 0 to 255.
    Raises:
        FileNotFoundError: Could not find `filename`
        ValueError: An error occurred while reading the video
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(filename)
    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_height, frame_width), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Use grayscale images
        v[count, :, :] = frame

    return v

In [ ]:
import tensorflow as tf


def videos_gen():
    for filename in filenames:
        traces = volumetracings_df.loc[filename]
        ed, es = int(traces.iloc[0]["Frame"]), int(traces.iloc[-1]["Frame"])
        video = loadvideo(videos_dir + filename)
        yield video, ed, es


ds = tf.data.Dataset.from_generator(
    videos_gen,
    output_signature=(
        tf.RaggedTensorSpec(shape=(None, 112, 112), dtype=tf.uint8),
        tf.TensorSpec(shape=(), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32),
    ),
)
# ds = ds.prefetch(10)
#ds = ds.map(
#    lambda video, ed, es: tf.py_function(
#        lf.label_frames,
#        (video, ed, es),
#        tf.float32,
#    )
#)
ds = ds.as_numpy_iterator()
next(ds)
